In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from matplotlib.ticker import PercentFormatter
import kaplanmeier as km
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import seaborn as sns
from statannot import add_stat_annotation
from itertools import combinations 

# Read in TARGET data files

In [3]:
# Read in sample z-score data from 'data_mRNA_median_all_sample_Zscores.txt' into pandas dataframe
z_scores = pd.read_csv('neuroblastoma_gene_expression_analysis_files/nbl_target_2018_pub/data_mRNA_median_all_sample_Zscores.txt',  sep='\t')

In [4]:
#Read in patient and sample clinical data into pandas dataframe
clinical_patient = pd.read_csv('neuroblastoma_gene_expression_analysis_files/nbl_target_2018_pub/data_clinical_patient.txt',  sep='\t', error_bad_lines=False)
clinical_sample = pd.read_csv('neuroblastoma_gene_expression_analysis_files/nbl_target_2018_pub/data_clinical_sample.txt',  sep='\t', error_bad_lines=False)



In [5]:
#Trim patient and sample clinical data to get rid of extraneous headers
clinical_patient_trimmed = clinical_patient.iloc[4:,:]
clinical_sample_trimmed = clinical_sample.iloc[4:,:]

# Functions for stratifying data

In [7]:
#This function stratifies samples based on numerical information in the 'data_clinical_patient.txt' file
#Inputs: Column you want to stratify data on, numerical threshold you want to stratify by
#Output: 2-D array containing lists of samples in each stratified group
def stratifyNumericalPatientData(stratifyingColumn,threshold):
    
    #Stratify patients based threshold in stratifyingColumn
    group1_patients = clinical_patient_trimmed.loc[clinical_patient_trimmed[stratifyingColumn].astype(float) < threshold]['#Patient Identifier']
    group2_patients = clinical_patient_trimmed.loc[clinical_patient_trimmed[stratifyingColumn].astype(float) >= threshold]['#Patient Identifier']
    
    #Stratify samples based on patients in each group
    group1_samples = clinical_sample_trimmed.loc[clinical_sample_trimmed['#Patient Identifier'].isin(group1_patients)]['Sample Identifier']
    group2_samples = clinical_sample_trimmed.loc[clinical_sample_trimmed['#Patient Identifier'].isin(group2_patients)]['Sample Identifier']
    
    return [group1_samples, group2_samples]




In [8]:
#This function stratifies samples based on categorical information in the 'data_clinical_patient.txt' file
#Inputs: Column you want to stratify data on, a list of the names of the categories within the stratifyingColumn you want to stratify on
#Output: 2-D array containing lists of samples in each stratified group
def stratifyCategoricalPatientData(stratifyingColumn,categories):
    
    #Create 2-D array to be returned
    sample_array = []
    
    #Stratify patients based on categories in stratifyingColumn
    for category in categories:
        group_patients = clinical_patient_trimmed.loc[clinical_patient_trimmed[stratifyingColumn]==category]['#Patient Identifier']
        group_samples = clinical_sample_trimmed.loc[clinical_sample_trimmed['#Patient Identifier'].isin(group_patients)]['Sample Identifier']
        sample_array.append(group_samples)
    
    return sample_array
    
    
    

In [6]:
#This function stratifies samples based on categorical information in the 'data_clinical_sample.txt' file
#Inputs: Column you want to stratify data on, a list of the names of the categories within the stratifyingColumn you want to stratify on
#Output: 2-D array containing lists of samples in each stratified group
def stratifyCategoricalSampleData(stratifyingColumn,categories):
    
    #Create 2-D array to be returned
    sample_array = []
    
    #Statify samples based on categories in stratifyingColumn
    for category in categories:
        group_samples = clinical_sample_trimmed.loc[clinical_sample_trimmed[stratifyingColumn]==category]['Sample Identifier']
        sample_array.append(group_samples)
    
    return sample_array
    
    
    

In [10]:
#This function stratifies samples based on the expression of a gene of interest
#Inputs: gene of interest, lower z-score threshold, upper z-score threshold
#Outputs: 2-D array containing lists of samples in each stratified group
#Statifies data into two groups (below lowerThresh, and above upperThresh)
def stratifyDataOnGeneExpression_twoGroups(gene,lowerThresh,upperThresh,z_scores=z_scores):
    #Obtain the z-scores for the gene of interest
    gene_z_scores = z_scores.loc[z_scores['Hugo_Symbol']==gene].iloc[:,2:].transpose()
    #Find samples for which the z-scores are < lowerThresh
    lower_expression_samples = gene_z_scores.loc[gene_z_scores.iloc[:,0] < lowerThresh].index
    #Find samples for which the z-scores are > upperThresh
    higher_expression_samples = gene_z_scores.loc[gene_z_scores.iloc[:,0] > upperThresh].index
    return [lower_expression_samples, higher_expression_samples]

In [11]:
#This function stratifies samples based on the expression of a gene of interest
#Inputs: gene of interest, lower z-score threshold, upper z-score threshold
#Outputs: 2-D array containing lists of samples in each stratified group
#Stratifies data into three groups (low, intermediate, and high)
def stratifyDataOnGeneExpression_threeGroups(gene,lowerThresh,upperThresh):
    #Obtain the z-scores for gene of interest
    gene_z_scores = z_scores.loc[z_scores['Hugo_Symbol']==gene].iloc[:,2:].transpose()
    #Find samples for which the z-scores are < lowerThresh
    lower_expression_samples = gene_z_scores.loc[gene_z_scores.iloc[:,0] < lowerThresh].index
    #Find samples for which z-scores are between thresholds
    intermediate_z_scores = gene_z_scores.loc[gene_z_scores.iloc[:,0] > lowerThresh]
    intermediate_expression_samples = intermediate_z_scores.loc[intermediate_z_scores.iloc[:,0] < upperThresh].index
    #Find samples for which the z-scores are > upperThresh
    higher_expression_samples = gene_z_scores.loc[gene_z_scores.iloc[:,0] > upperThresh].index
    return [lower_expression_samples, intermediate_expression_samples, higher_expression_samples]

# Function for comparing expression of a single gene between multiple groups

In [7]:
#This function compares the expression of a gene of interest between multiple groups
#Inputs: Hugo_Symbol of gene of interest, 2-D array containing lists of samples in each groups (output of functions above),
#title of graph, list of labels for each category (for labelling graphs), and the type of statistical test
#you want to use to compare groups (see https://github.com/webermarcolivier/statannot for the types of tests available)
#Outputs: boxplot of gene expression compared between groups, results of statistical test, summary statistics
def compareExpression(gene,groups,title,categoryLabels, testType):
    #Find z-scores for specified gene for each group, and store in list
    gene_expression_values = []
    for group in groups:
        group_columns = ['Hugo_Symbol','Entrez_Gene_Id'] + list(set(group).intersection(z_scores.columns))
        group_z_scores = z_scores[group_columns]
        group_gene_expression_df = group_z_scores.loc[group_z_scores['Hugo_Symbol']==gene]
        group_gene_expression_values = group_gene_expression_df.iloc[0,2:].values.astype(float)
        gene_expression_values.append(group_gene_expression_values)
    
    #Update category labels to include sample size
    updated_categoryLabels = []
    for i in range(len(categoryLabels)):
        updated_categoryLabels.append(categoryLabels[i] + ' (n = ' + str(len(gene_expression_values[i])) + ')')
    
    #Make dataframe of gene expression values
    gene_expression_values_df = pd.DataFrame(index=range(max([len(i) for i in gene_expression_values])))
    for i in range(len(categoryLabels)):
        gene_expression_values_df[categoryLabels[i]] = pd.Series(gene_expression_values[i])
    
    #Box plot comparing expression between groups
    ax2 = ax2 = sns.boxplot(data=gene_expression_values)
    ax2.set_ylabel('z-scores')
    ax2.set_xticklabels(updated_categoryLabels)
    ax2.set_title(title)
    ax2.set_ylim(-5,5)
    ax2, test_results = add_stat_annotation(ax2, data=gene_expression_values_df,
                    box_pairs=list(combinations(categoryLabels,2)),
                    test=testType, text_format='simple', loc='inside', verbose=2, comparisons_correction=None)
    plt.show()
    
    #Print median, mean, and 95% CI of z-scores for each group
    for i in range(len(gene_expression_values)):
        print(categoryLabels[i] + ": median = " + str(np.median(gene_expression_values[i]))
         + ", mean = " + str(np.mean(gene_expression_values[i]))
         + ", 95% CI = " + str(st.norm.interval(alpha=0.95, loc=np.mean(gene_expression_values[i]), scale=st.sem(gene_expression_values[i]))))
        

    
    
    

# Function for comparing EFS between multiple groups

In [16]:
#This function compares the Event-Free Survival (EFS) between multiple groups
#Inputs: 2-D array containing lists of samples in each groups (output of functions above),
#title of graph, list of labels for each group, type of test you want to use to compare groups
#Outputs: boxplot of EFS compared between groups, summary statistics
def compareEFS(groups, title, labels, testType):
    #Get EFS times for each group
    EFS = []
    for group in groups:
        patients = clinical_sample_trimmed.loc[clinical_sample_trimmed['Sample Identifier'].isin(group)]['#Patient Identifier']
        group_EFS = clinical_patient_trimmed.loc[clinical_patient_trimmed['#Patient Identifier'].isin(patients)]['EFS Time'].astype(float).dropna()
        EFS.append(group_EFS)
    
    #Update  labels to include sample size
    updated_labels = []
    for i in range(len(labels)):
        updated_labels.append(labels[i] + ' (n = ' + str(len(EFS[i])) + ')')
        
    #Make dataframe of EFS times
    EFS_df = pd.DataFrame(index=range(max([len(i) for i in EFS])))
    for i in range(len(labels)):
        EFS_df[labels[i]] = pd.Series(EFS[i].values)
    
    #Boxplot comparing EFS time of each group
    ax = sns.boxplot(data=EFS)
    ax.set_ylabel('EFS (Days)')
    ax.set_xticklabels(updated_labels)
    ax.set_title(title)
    ax, test_results = add_stat_annotation(ax, data=EFS_df,
                    box_pairs=list(combinations(labels,2)),
                    test=testType, text_format='simple', loc='inside', verbose=2, comparisons_correction=None)
    plt.show()
    
    #Print median, mean, and 95% CI of z-scores for each group
    for i in range(len(EFS)):
        print(labels[i] + ": median = " + str(np.median(EFS[i]))
         + ", mean = " + str(np.mean(EFS[i]))
         + ", 95% CI = " + str(st.norm.interval(alpha=0.95, loc=np.mean(EFS[i]), scale=st.sem(EFS[i]))))
        
        

# Function for making Kaplan Meier curves comparing multiple groups

In [ ]:
#This function plots a Kaplan Meier curve based on event-free survival of multiple groups
#Inputs: 2-D array containing lists of samples in each groups (output of functions above),
#title of graph, list of string labels for each group
#Output: Kaplan Meier curve
#This function utilizes the kaplanmeier python package (make sure to cite): https://pypi.org/project/kaplanmeier/
#This function is capable of comparing multiple groups, unlike original function
def kaplanmeierEFS(groups, title, labels):
    #Get EFS times for each group
    EFS = []
    for group in groups:
        patients = clinical_sample_trimmed.loc[clinical_sample_trimmed['Sample Identifier'].isin(group)]['#Patient Identifier']
        group_EFS = clinical_patient_trimmed.loc[clinical_patient_trimmed['#Patient Identifier'].isin(patients)]['EFS Time'].astype(float).dropna()
        group_EFS = group_EFS/365
        EFS.append(group_EFS)
    
    #Get event status for each group
    event_status = []
    i=0
    for group in groups:
        patients = clinical_sample_trimmed.loc[clinical_sample_trimmed['Sample Identifier'].isin(group)]['#Patient Identifier']
        event_string = clinical_patient_trimmed.loc[clinical_patient_trimmed['#Patient Identifier'].isin(patients)]['First Event']
        for index in (set(event_string.index)-set(EFS[i].index)):
            del event_string[index]
        event_num=[]
        for status in event_string:
            if(status == 'Censored'):
                event_num.append(0)
            else:
                event_num.append(1)
        event_status.append(event_num)
        i+=1
    
    #Create dataframe containing EFS Time and Event Status for each group
    group_dfs = []
    for i in range(len(groups)):
        group_df = pd.DataFrame()
        group_df['EFS_time'] = EFS[i]
        group_df['Event_Status'] = event_status[i]
        group_df['Group'] = labels[i]
        group_dfs.append(group_df)
    
    #Combine dataframes
    combined_df = pd.concat(group_dfs).dropna()
    
    #Use kaplanmeier Python package to plot Kaplan-Meier survival curve
    for i in range(len(groups)):
        kmf = KaplanMeierFitter(alpha=1)
        kmf.fit(group_dfs[i]['EFS_time'], group_dfs[i]['Event_Status'], label=labels[i])
        ax = kmf.plot()
        #Print median survival time and probability of 5yr survival
        print(labels[i] + ' median survival time = ' + str(kmf.median_survival_time_))
        print(labels[i] + ' probability of 5yr EFS = ' + str(kmf.predict(5)))
    ax.set_title(title)
    ax.set_ylabel('EFS proportion')
    ax.set_xlabel('Years')
    ax.set_ylim([0,1])
    ax.legend(loc='lower left')
    plt.show()
    
    #logrank_test
    for combo in list(combinations(labels,2)):
        group1=combined_df[combined_df['Group']==combo[0]]
        group2=combined_df[combined_df['Group']==combo[1]]
        T=group1['EFS_time']
        E=group1['Event_Status']
        T1=group2['EFS_time']
        E1=group2['Event_Status']
        results=logrank_test(T,T1,event_observed_A=E, event_observed_B=E1)
        print('Log Rank Test between ' + combo[0] + ' and ' + combo[1])
        results.print_summary()
        print('p-value = ' + str(results.p_value))
    
    

# Function for making a scatter plot of z-scores between two genes

In [58]:
#This function makes a scatter plot for comparing the z-scores of two genes across all TARGET samples
#Inputs: (1) gene to be plotted on x-axis, (2) gene to be plotted on y-axis, and (3) an optional parameter
#that allows you to specify the dataframe of z-scores you want to use (this is set to the dataframe of
#z-scores for all TARGET samples by default, but if you want to only analyze a subset of the samples, you
#should specify your own z-score dataframe)
def geneScatter(gene1, gene2,z_scores=z_scores):
    #Get all z-scores for each gene
    gene1_z_scores = z_scores.loc[z_scores['Hugo_Symbol']==gene1].iloc[:,2:].transpose()
    gene2_z_scores = z_scores.loc[z_scores['Hugo_Symbol']==gene2].iloc[:,2:].transpose()
    
    #Make scatter plot that includes a line of best fit
    plt.scatter(gene1_z_scores,gene2_z_scores)
    plt.xlabel(gene1 + ' z-score')
    plt.ylabel(gene2 + ' z-score')
    m,b = np.polyfit(gene1_z_scores.iloc[:,0].values,gene2_z_scores.iloc[:,0].values,1)
    plt.plot(gene1_z_scores, m*(gene1_z_scores.iloc[:,0].values) + b, 'orange')
    plt.show()
    
    #Calculate pearson's correlation coefficient and the p-value from the pearson correlation test
    corr,pvalue = st.pearsonr(gene1_z_scores.iloc[:,0].tolist(),gene2_z_scores.iloc[:,0].tolist())
    print('Pearson correlation coefficient = ' + str(corr))
    print('p-value = ' + str(pvalue))
